In [30]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, roc_auc_score,roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
import time
import numpy as np 
import pandas as pd
import os

In [31]:
og=pd.read_csv('/home/vijay/Rutgers/course_work/553/project/TCPDUMP_and_CICFlowMeter/csv/dump_ISCX.csv')

In [32]:
og.drop(['Flow ID','Src IP','Src Port','Dst IP','Protocol','Timestamp'], inplace=True, axis=1)
og.rename(columns = {'Dst Port':' Destination Port', 'Total Bwd packets':'Total Backward Packets','Packet Length Min':' Min Packet Length','Packet Length Max':' Max Packet Length','CWR Flag Count':' CWE Flag Count','Fwd Segment Size Avg':' Avg Fwd Segment Size','Bwd Segment Size Avg':' Avg Bwd Segment Size','Fwd Bytes/Bulk Avg':'Fwd Avg Bytes/Bulk'}, inplace = True)
og.rename(columns={'Fwd Packet/Bulk Avg':' Fwd Avg Packets/Bulk','Fwd Bulk Rate Avg':' Fwd Avg Bulk Rate','Bwd Bytes/Bulk Avg':' Bwd Avg Bytes/Bulk','Bwd Packet/Bulk Avg':' Bwd Avg Bytes/Bulk','Bwd Bulk Rate Avg':'Bwd Avg Bulk Rate','FWD Init Win Bytes':'Init_Win_bytes_forward','Bwd Init Win Bytes':' Init_Win_bytes_backward','Fwd Act Data Pkts':' act_data_pkt_fwd'}, inplace=True)
og.rename(columns={'Fwd Seg Size Min':' min_seg_size_forward'}, inplace=True)

In [33]:
og

,Destination Port,Flow Duration,Total Fwd Packet,Total Backward Packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,283,2,0,63.0,0.0,39.0,24.0,31.5,10.606601717798213,...,32,0,0,0,0,1.683406211556262E15,0.0,1.683406211556262E15,1.683406211556262E15,NeedManualLabel
1,33512,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0,0,0,0,1.683406211562064E15,0.0,1.683406211562064E15,1.683406211562064E15,NeedManualLabel
2,Dst Port,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
3,443,9178,4,3,215.0,590.0,159.0,0.0,53.74999999999999,74.96832664532403,...,32,0,0,0,0,1.68340621747807E15,0.0,1.68340621747807E15,1.68340621747807E15,NeedManualLabel
4,443,37707,4,8,2893.0,441.0,2691.0,0.0,723.25,1313.6728093402862,...,32,0,0,0,0,1.683406217477992E15,0.0,1.683406217477992E15,1.683406217477992E15,NeedManualLabel
5,443,66222,2,2,69.0,71.0,69.0,0.0,34.5,48.79036790187178,...,32,0,0,0,0,1.683406220955986E15,0.0,1.683406220955986E15,1.683406220955986E15,NeedManualLabel
6,5353,6143015,2,0,1675.0,0.0,1266.0,409.0,837.5,605.9905114768712,...,8,0,0,0,0,8.417031106528955E14,1.1903479458894118E15,1.683406215162776E15,6143015.0,NeedManualLabel
7,443,7371,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0,0,0,0,1.683406213574222E15,0.0,1.683406213574222E15,1.683406213574222E15,NeedManualLabel


In [35]:
og[" Destination Port"]

0         443
1       33512
2    Dst Port
3         443
4         443
5         443
6        5353
7         443
Name:  Destination Port, dtype: object

In [36]:
for i in range(len(og)):
    if og[" Destination Port"][i] == 'Dst Port':
        og.drop(i,inplace=True)

In [37]:
og =  og.drop_duplicates(keep="first")
og.reset_index(drop=True,inplace=True)

In [38]:
og.columns =  og.columns.str.strip()

In [39]:
og = og[~og.isin([np.nan, np.inf, -np.inf]).any(1)]
og.dropna(inplace=True)

TypeError: NDFrame._add_numeric_operations.<locals>.any() takes 1 positional argument but 2 were given

In [9]:
x_test=og.drop(["Label"],axis=1)

In [10]:
x_train = pd.read_csv('xtrain_dev.csv') 

In [22]:
for i in range(len(x_test)):
    if x_test["Destination Port"][i] == 'Dst Port':
        x_test.drop(i,inplace=True)

In [47]:
x_test["Total Length of Fwd Packet"][3]

'215.0'

In [41]:
for i in range(len(x_test)):
    if x_test["Total Length of Fwd Packet"][i] == '63.0':
        # x_test.drop(i,inplace=True)
        print('FOUND')
        print(i)


FOUND
0


KeyError: 2

In [19]:
for i in x_test.columns:
    print(i)

Destination Port
Flow Duration
Total Fwd Packet
Total Backward Packets
Total Length of Fwd Packet
Total Length of Bwd Packet
Fwd Packet Length Max
Fwd Packet Length Min
Fwd Packet Length Mean
Fwd Packet Length Std
Bwd Packet Length Max
Bwd Packet Length Min
Bwd Packet Length Mean
Bwd Packet Length Std
Flow Bytes/s
Flow Packets/s
Flow IAT Mean
Flow IAT Std
Flow IAT Max
Flow IAT Min
Fwd IAT Total
Fwd IAT Mean
Fwd IAT Std
Fwd IAT Max
Fwd IAT Min
Bwd IAT Total
Bwd IAT Mean
Bwd IAT Std
Bwd IAT Max
Bwd IAT Min
Fwd PSH Flags
Bwd PSH Flags
Fwd URG Flags
Bwd URG Flags
Fwd Header Length
Bwd Header Length
Fwd Packets/s
Bwd Packets/s
Min Packet Length
Max Packet Length
Packet Length Mean
Packet Length Std
Packet Length Variance
FIN Flag Count
SYN Flag Count
RST Flag Count
PSH Flag Count
ACK Flag Count
URG Flag Count
CWE Flag Count
ECE Flag Count
Down/Up Ratio
Average Packet Size
Avg Fwd Segment Size
Avg Bwd Segment Size
Fwd Avg Bytes/Bulk
Fwd Avg Packets/Bulk
Fwd Avg Bulk Rate
Bwd Avg Bytes/Bulk
B

In [23]:
for i in x_test.columns:
    x_test = x_test.astype({i: 'int'})

ValueError: invalid literal for int() with base 10: '63.0': Error while type casting for column 'Total Length of Fwd Packet'

In [15]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
# x_test = scaler.fit_transform(x_test)

/home/vijay/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'Dst Port'